In [3]:
import pandas as pd

# 1. 讀取 CSV時強制所有欄位為文字 (dtype=str)
# 這樣可以防止 '000' 被 pandas 自動轉成數字 0
df = pd.read_csv('GoogleMaps_V3_2025-12-28.csv', on_bad_lines='skip', dtype=str)

def fix_shifted_rows(row):
    # --- 情況 A：位移了兩格 (有兩個逗號，資料跑到 Unnamed: 6) ---
    if pd.notna(row.get('Unnamed: 6')):
        p1 = str(row['價格帶'])
        p2 = str(row['圖片網址'])
        p3 = str(row['Unnamed: 5'])
        
        # 把三段接回來 (例如: "超過 ¥1" + "," + "000" + "," + "000")
        row['價格帶'] = f"{p1},{p2},{p3}"
        # 真正的圖片網址在 Unnamed: 6
        row['圖片網址'] = row['Unnamed: 6']

    # --- 情況 B：位移了一格 (這是一般情況，資料跑到 Unnamed: 5) ---
    elif pd.notna(row.get('Unnamed: 5')):
        p1 = str(row['價格帶'])
        p2 = str(row['圖片網址'])
        
        # 把兩段接回來 (例如: "¥10" + "," + "000")
        row['價格帶'] = f"{p1},{p2}"
        # 真正的圖片網址在 Unnamed: 5
        row['圖片網址'] = row['Unnamed: 5']
        
    return row

# 應用修復
df_cleaned = df.apply(fix_shifted_rows, axis=1)

# 選取並重整正確的欄位
final_columns = ['店名', '評價', '評論數', '價格帶', '圖片網址']
df_final = df_cleaned[final_columns]

# 檢查修復結果 (看看 L'évo 這種高價位餐廳是否正常了)
print("修復範例 (L'évo):")
print(df_final[df_final['店名'].str.contains("L'évo", na=False)])

# 存檔
df_final.to_csv('GoogleMaps_Final_Cleaned_V2.csv', index=False, encoding='utf-8-sig')
print("\n✅ 檔案已修復並儲存為 GoogleMaps_Final_Cleaned_V2.csv")

修復範例 (L'évo):
      店名   評價  評論數       價格帶  \
4  L'évo  4.6  244  超過 ¥10,0   

                                                圖片網址  
4  https://lh3.googleusercontent.com/gps-cs-s/AG0...  

✅ 檔案已修復並儲存為 GoogleMaps_Final_Cleaned_V2.csv
